In [1]:
import pandas as pd
import streamlit as st
import plotly.express as px
from PIL import Image
import os
import camelot
import matplotlib.pyplot as plt
import numpy as np
import base64
import io

def parseData(file):
        tables = camelot.read_pdf(f'statements/{file}', flavor='stream', pages='all')
        concat = tables[0].df

        ## concatenate all tables into one
        for i in range(1, len(tables)):
            concat = pd.concat([concat, tables[i].df])

        ## filter out nonsensical data
        concat = concat[~concat.apply(lambda row: row.astype(str).str.contains('about:blank')).any(axis=1)]

        ## reset index of df
        concat = concat.reset_index(drop=True)

        if concat.shape[-1] == 7:
            concat = concat.drop(concat.columns[-1], axis=1)

        ## rename columns
        concat.columns = ['Date', 'Code', 'Reference', 'Debit', 'Credit', 'Extra']

        ## handle bad formatting
        for row in range(len(concat)):
            value = concat.at[row, 'Credit']
            extraVal = concat.at[row, 'Extra']
            
            if value != '':
                concat.at[row, 'Debit'] = value
                concat.at[row, 'Credit'] = ''
            
            if extraVal != '':
                concat.at[row, 'Credit'] = extraVal
                concat.at[row, 'Extra'] = ''
                
        ## drop useless column
        concat = concat.drop(columns=['Extra'])

        firstCell = concat.at[0, 'Date']
        secondCell = concat.at[1, 'Debit']
        thirdCell = concat.at[2, 'Debit']
        fourthCell = concat.at[3, 'Debit']

        if firstCell.split(' ')[0] == 'Transaction':
            concat = concat.drop([0])
            
        if secondCell == 'Download':
            concat = concat.drop([1])
            
        if thirdCell == 'Debit':
            concat = concat.drop([2])
            
        if fourthCell == '(Withdrawal)':
            concat = concat.drop([3])
            
        concat = concat.reset_index(drop=True)
        concat['Type'] = ''
        concat = concat.reindex(columns=['Date', 'Code', 'Type', 'Reference', 'Debit', 'Credit'])
        for a in range(len(concat)):
            code = concat.at[a, 'Code']
            steps = 0
            
            if code != '':
                addArr = []
                index = a + 1
                
                while True:
                    if index > len(concat)-1:
                        break
                        
                    check = concat.at[index, 'Code']
                    if check == '':
                        addArr.append(concat.at[index, 'Reference'])
                        steps += 1
                        index += 1
                    else:
                        break
                    
                for step in range(steps):
                    if concat.at[a, 'Reference'][-1] == '-' or concat.at[a, 'Reference'][-1].isdigit() == True:
                        concat.at[a, 'Reference'] += concat.at[a+step+1, 'Reference']
                    else:
                        concat.at[a, 'Reference'] += ' ' + concat.at[a+step+1, 'Reference']
                        
                    concat.at[a+step+1, 'Reference'] = ''

        ## remove all empty rows
        concat = concat.loc[~(concat == '').all(axis=1)]
        concat = concat.reset_index(drop=True)

        if concat.at[concat.shape[0]-1, 'Debit'] != '' and concat.at[concat.shape[0]-1, 'Credit'] != '':
            DrBal = concat.at[concat.shape[0]-1, 'Debit']
            CrBal = concat.at[concat.shape[0]-1, 'Credit']
            concat = concat.drop(concat.index[-1])

        for TYPE in range(len(concat)):
            code = concat.at[TYPE, 'Code']
            
            if code == 'MST':
                concat.at[TYPE, 'Type'] = 'Card Transaction'
                
            if code == 'ITR' or code == 'ICT':
                concat.at[TYPE, 'Type'] = 'Funds Transfer'
                
            if code == 'POS':
                concat.at[TYPE, 'Type'] = 'Point-Of-Sale'
                
            if code == 'INT':
                concat.at[TYPE, 'Type'] = 'Interest Earned'

        concat = concat.reindex(index=concat.index[::-1])
        concat.reset_index(drop=True)

        concat = concat.loc[~((concat['Date'] == 'Date') & (concat['Code'] == 'Code'))]
        concat = concat.loc[~((concat['Debit'] == 'Download') & (concat['Credit'] == 'Print'))]
        concat = concat.loc[~((concat['Debit'] == '(Withdrawal)') & concat['Credit'] == '(Deposit)')]
        concat = concat.replace('', np.nan)
        concat = concat.dropna(thresh=len(concat.columns) - 3)
        concat = concat.reset_index(drop=True)
        concat = concat.loc[~((concat['Date'].str.contains('Transaction History')))]
        concat = concat.reset_index(drop=True)
    
        return concat

In [2]:
directory = os.getcwd()
path = directory + '\statements'
filenames = os.listdir(path)
concatList = []

for pdf in range(len(filenames)):
    concat = parseData(filenames[pdf])
    concatList.append(concat)

concat = pd.concat(concatList, axis=0)
concat = concat.reset_index(drop=True)

2023-02-24T23:13:14 - INFO - Processing page-1
2023-02-24 23:13:14.694 INFO    camelot: Processing page-1
2023-02-24T23:13:15 - INFO - Processing page-2
2023-02-24 23:13:15.215 INFO    camelot: Processing page-2
2023-02-24T23:13:15 - INFO - Processing page-3
2023-02-24 23:13:15.801 INFO    camelot: Processing page-3
2023-02-24T23:13:16 - INFO - Processing page-4
2023-02-24 23:13:16.271 INFO    camelot: Processing page-4
2023-02-24T23:13:16 - INFO - Processing page-5
2023-02-24 23:13:16.743 INFO    camelot: Processing page-5
2023-02-24T23:13:17 - INFO - Processing page-6
2023-02-24 23:13:17.308 INFO    camelot: Processing page-6
2023-02-24T23:13:17 - INFO - Processing page-7
2023-02-24 23:13:17.794 INFO    camelot: Processing page-7
2023-02-24T23:13:17 - INFO - Processing page-8
2023-02-24 23:13:17.856 INFO    camelot: Processing page-8
2023-02-24T23:13:21 - INFO - Processing page-1
2023-02-24 23:13:21.473 INFO    camelot: Processing page-1
2023-02-24T23:13:21 - INFO - Processing page-2

In [5]:
concat['Balance'] = ''
final = 204.47
concat.at[concat.shape[0]-1, 'Balance'] = 'S$' + str(204.47)
index = concat.shape[0]-2

totalDr = totalCr = 0
for agg in range(len(concat)):

    if concat.at[index, 'Debit'] != '':
        if isinstance(concat.at[index, 'Debit'], float) == False:
            update = final - float(concat.at[index, 'Debit'][2:])
            totalDr += float(concat.at[index, 'Debit'][2:])
            concat.at[index, 'Balance'] = 'S$' + str(round(update, 2))

    if concat.at[index, 'Credit'] != '':
        if isinstance(concat.at[index, 'Credit'], float) == False:
            update = final + float(concat.at[index, 'Credit'][2:])
            totalCr += float(concat.at[index, 'Credit'][2:])
            concat.at[index, 'Balance'] = 'S$' + str(round(update, 2))


    if index != 0:
        index -= 1

In [6]:
concat

,Date,Code,Type,Reference,Debit,Credit,Balance
0,28 Nov 2022,ICT,Funds Transfer,FAST / PayNow Transfer Incoming PayNow Ref 394...,NaN,S$80.00,S$284.47
1,29 Nov 2022,MST,Card Transaction,Debit Card Transaction MCDONALD'S (PLSQ) SI NG...,S$21.75,NaN,S$182.72
2,29 Nov 2022,MST,Card Transaction,Debit Card Transaction GOJEK 90 10 26NOV 4628-...,S$19.40,NaN,S$185.07
3,29 Nov 2022,MST,Card Transaction,Debit Card Transaction GOJEK 90 10 27NOV 4628-...,S$28.70,NaN,S$175.77
4,30 Nov 2022,MST,Card Transaction,Debit Card Transaction GENKI SUSHI-SPC SI NG 2...,S$43.20,NaN,S$161.27
...,...,...,...,...,...,...,...
229,19 Feb 2023,MST,Card Transaction,Debit Card Transaction ORDER.PLAC* TONKOTSUKI ...,S$43.60,NaN,S$160.87
230,19 Feb 2023,MST,Card Transaction,Debit Card Transaction ORDER.PLAC* TONKOTSUKI ...,S$4.65,NaN,S$199.82
231,19 Feb 2023,MST,Card Transaction,Debit Card Transaction AUNTIE ANNE'S-PAYA LEB ...,S$7.20,NaN,S$197.27
232,21 Feb 2023,MST,Card Transaction,Debit Card Transaction GRAB SI NG 18FEB 4628-4...,S$19.10,NaN,S$185.37


In [43]:
def totalDrCr():
    sumDF = concat.copy()
    sumDF['Debit'] = sumDF['Debit'].str[2:]
    sumDF['Credit'] = sumDF['Credit'].str[2:]

    sumDF['Debit'] = sumDF['Debit'].astype(float)
    sumDF['Credit'] = sumDF['Credit'].astype(float)
    sumDF['Date'] = sumDF['Date'].str[-4:]

    years = sumDF['Date'].unique()

    color_map = {'Debit': 'rgb(228,26,28)', 'Credit': 'rgb(55,126,184)'}

    grouped_df = sumDF.groupby('Date').agg({'Debit': 'sum', 'Credit': 'sum'}).reset_index()

    # Create a bar chart of total debit and credit amounts by year
    fig = px.bar(grouped_df, x='Date', y=['Debit', 'Credit'], barmode='group', color_discrete_map=color_map)

    # Add axis labels and title
    fig.update_layout(
        xaxis_title="Year",
        yaxis_title="Amount",
        title="Total Debit and Credit Amounts by Year"
)


    return fig

totalDrCr()